# Lab 11 B - Clasificacion de video con RNNs (CNN + LSTM)

El objetivo de este laboratorio es entrenar un clasificador de videos utilizando una red neuronal recurrente (RNN). Para ello, se utilizará parte del dataset [UCF101](https://www.crcv.ucf.edu/data/UCF101.php) que contiene mas de 13000 videos clasificados en 101 clases. Para los fines del laboratorio se seleccionaron solo 3 clases del dataset (Basketball, Biking, Bowling).

![](https://www.crcv.ucf.edu/data/UCF101/UCF101.jpg)

## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab11"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)


Importar librerías

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

## Preparacion del dataset

En el interior de la carpeta dataset nos encontramos una carpeta por cada clase. En cada una de estas carpetas depositamos los videos representativos de su respectiva clase.

In [ ]:
data_dir = 'datasets/video-classification-dataset'
classes = os.listdir(data_dir)
num_classes = len(classes)
print(classes)

Definimos los parametros de carga:

![](images/segment.png)

In [ ]:
img_height, img_width = 50, 50
segment_lenght = 10  # Número de frames a extraer por segmento
segment_stride = 5  # Desplazamiento entre segmentos

Definimos una funcion para cargar el dataset a partir de los parametros mencionados:

In [ ]:
def load_video_classification_dataset(data_dir, img_height, img_width, segment_lenght, segment_stride):
    X, y = [], []
    for label, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for video_name in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_name)
            cap = cv2.VideoCapture(video_path)
            frames = []
            success, frame = cap.read()
            while success:
                frame = cv2.resize(frame, (img_width, img_height))
                frames.append(frame)
                success, frame = cap.read()
            cap.release()

            # Extraer segmentos de frames
            num_segments = (len(frames) - segment_lenght) // segment_stride + 1
            for i in range(num_segments):
                start = i * segment_stride
                end = start + segment_lenght
                segment = frames[start:end]
                if len(segment) == segment_lenght:
                    X.append(segment)
                    y.append(label)

    X = np.array(X)
    y = to_categorical(y, num_classes)
    return X, y

Cargamos el dataset y aplicamos shuffling y validation split:

In [ ]:
# Cargar y preprocesar los datos
X, y = load_video_classification_dataset(data_dir, img_height, img_width, segment_lenght, segment_stride)

# Mezclar los datos antes de dividirlos en entrenamiento y validación
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Arquitectura del modelo y entrenamiento

La arquitectura que se presenta a continuacion podemos descomponerla en 3 partes:


1.   Capas convolucionales: Extraccion de caracteristicas espaciales.
2.   Capa recurrente: Extraccion de caracteristicas temporales.
3.   Capas de salida: Juntan las caracteristicas 1, 2 y producen la prediccion de clase.



Nota: La capa **TimeDistributed** se usa para aplicar las capas convolucionales a cada frame del video de forma independiente antes de pasar las características extraídas a la capa LSTM, que luego puede capturar las dependencias temporales entre estos frames.


In [ ]:
model_cnlst = Sequential()
# Capas convolucionales
model_cnlst.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu'), input_shape=(segment_lenght, img_height, img_width, 3)))
model_cnlst.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(Conv2D(16, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(BatchNormalization()))
model_cnlst.add(TimeDistributed(Flatten()))
model_cnlst.add(Dropout(0.5))
# Capa recurrente
model_cnlst.add(LSTM(32, return_sequences=False, dropout=0.5))
# Capas de salida
model_cnlst.add(Dense(64, activation='relu'))
model_cnlst.add(Dense(32, activation='relu'))
model_cnlst.add(Dropout(0.5))
model_cnlst.add(Dense(num_classes, activation='softmax'))
# Resumen del modelo
model_cnlst.summary()

Corremos el entrenamiento con early stopping y model checkpoints:

In [ ]:
# Compilar el modelo
model_cnlst.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Definir callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Entrenar el modelo con callbacks
history = model_cnlst.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=8,
                          callbacks=[early_stopping, model_checkpoint])

# Evaluación del modelo en el conjunto de validación
val_loss, val_accuracy = model_cnlst.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")


Analizamos los resultamos:

In [ ]:
# Visualización de la precisión y pérdida durante el entrenamiento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()